# __Brough Lecture Notes: GARCH Models - Estimation via MLE__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on the excellent monograph [Introduction to Python for Econometrics, Statistics, and Data Analysis](https://www.kevinsheppard.com/images/b/b3/Python_introduction-2016.pdf) by the econometrician Kevin Sheppard of Oxford Univeristy. Many thanks to Dr. Sheppard for making his lecture material publically available. 

<br>

## Estimating GARCH Models

The standard procedure to fit GARCH models to historical returns time series data is to implement a numerical _maximum likelihood estimation_ (MLE) method. 

<br>

__NB:__ though [Bayesian](https://www.springer.com/us/book/9783540786566) methods have been shown to be superior!

<br>

The typical setup is as follows: 

* Continuously compounded returns are assumed to have a conditionally normal distribution $N(0, \sigma_{t})$

* We can estimate the GARCH parameter weights via a numerical optimization routine such as Nelder-Mead or Newton-Raphson.

* That is, the numerical routine searches for the parameter values that maximizes the value of the likelihood function.

<br>

Under the normality assumption the probility density of $\epsilon_{t}$, conditional on $\sigma_{t}$, is 

<br>

$$
f(\epsilon | \sigma_{t}) = \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}}
$$

<br>

Since the $\epsilon_{t}$ are conditionally independent, the probability of observing the actual returns that are observed is  the product of the probabilities, this is given by the likelihood function:

<br>

$$
\prod\limits_{t=1}^{T} f(\epsilon_{t} | \sigma_{t}) = \prod\limits_{t=1}^{T} \left( \frac{1}{\sqrt{2\pi \sigma_{t}}} e^{-0.5  \frac{\epsilon_{t}^{2}}{\sigma_{t}}} \right)
$$

<br>

For the GARCH(1,1) model, $\sigma_{t}$ is a function of $\omega$, $\alpha$, and $\beta$. The MLE will select values for these parameters $\hat{\omega}$, $\hat{\alpha}$, and $\hat{\beta}$ - that maximize the value of the probability of observing the returns we actually historically did observe. 

<br>

Typically, it is easiest to maximize the value of the log-likelihood function as follows: 

<br>

$$
\sum\limits_{t=1}^{T} \left[ -0.5 \ln{(\sigma_{t})} - 0.5 \frac{\epsilon_{t}^{2}}{\sigma_{t}}\right]
$$

We can omit the term $-0.5 \ln{(2 \pi)}$ since it does not affect the solution. Though sometimes it is left in. 

<br>

We can implement this MLE estimation in Python by utilizing the [optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) module in [Scipy](https://docs.scipy.org/doc/scipy/reference/index.html), which contains a host of numerical optimization routines. 

<br>

First, we need to define a function to implement the log-likelihood function.

<br>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

import seaborn
from numpy import size, log, exp, pi, sum, diff, array, zeros, diag, mat, asarray, sqrt, copy
from numpy.linalg import inv

from scipy.optimize import fmin_slsqp

In [11]:
def garch_likelihood(params, data, sigma2, out=None):
    mu = params[0]
    omega = params[1]
    alpha = params[2]
    beta = params[3]
    
    T = size(data, 0)
    eps = data - mu
    
    for t in range(1, T):
        sigma2[t] = omega + alpha * eps[t-1]**2 + beta * sigma2[t-1]
        
    lls = 0.5 * (log(2 * pi) + log(sigma2) + eps**2/sigma2)
    ll = sum(lls)
    
    if out is None:
        results = ll
    else:
        results = (ll, lls, copy(sigma2))
        
    return results

<br>

We should begin with data simulated from the model as a first-run test case. So, let's import the simulation code.

<br>

In [12]:
## Set GARCH parameters (these might come from an estimated model)
mu = log(1.15) / 252
w = 10.0**-6
a = 0.085
b = 0.905

In [13]:
def simulate_garch(parameters, numObs):
    
    ## extract the parameter values
    mu = parameters[0]
    w = parameters[1]
    a = parameters[2]
    b = parameters[3]
    
    ## initialize arrays for storage
    z = np.random.normal(size=(numObs + 1))
    q = zeros((numObs + 1))
    r = zeros((numObs + 1))
    
    ## fix initial values 
    q[0] = w / (1.0 - a - b)
    r[0] = mu + z[0] * sqrt(q[0])
    e = (r[0] - mu) 
    
    ## run the main simulation loop
    for t in range(1, numObs + 1):
        q[t] = w + a * (e * e) + b * q[t-1]
        r[t] = mu + z[t] * sqrt(q[t])
        e = (r[t] - mu) 
        
    ## return a tuple with both returns and conditional variances
    return (r, q)

In [14]:
## number of trading days per year
numObs = 252

## daily continuously compounded rate of return correpsonding to 15% annual
mu = log(1.15) / 252

## drift and GARCH(1,1) parameters in an array
params = array([mu, w, a, b])

## run the simulation
r, s = simulate_garch(params, numObs)

<br>

Okay, now we will set up the estimation of the model on this simulated data. The goal is to see if the numerical search routine embedded in the maximum likelihood estimation
can recapture the preset parameter weights given above. 

<br>

It is important to test our software with a known scenario like this before we take the model to real-world data first as a sanity check. Otherwise we could end up chasing our tails for hours and hours without effect. 

<br>

The numerical algorithm requires good starting values as initial conditions for the search to begin. Here we will follow Sheppard. (Notice that this feels "prior" like, as in the Bayesian sense) A more data-based way to do this is to use some kind of grid search to find values that have "small" log-likelihood values. 

<br>

In [33]:
#begVals = array([r.mean(), r.var() * .01, .09, .90])
begVals = array([mu, w, a, b])

In [34]:
finfo = np.finfo(np.float64)
bounds = [(-10 * r.mean(), 10 * r.mean()), (finfo.eps, 2 * r.var()), (0.0, 1.0), (0.0, 1.0)]

estimates = fmin_slsqp(garch_likelihood, begVals, bounds=bounds, args=(r,s))

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -893.5303730848248
            Iterations: 7
            Function evaluations: 25
            Gradient evaluations: 3


In [35]:
estimates

array([4.95995571e-04, 2.38208122e-06, 8.35140525e-02, 8.89178546e-01])

In [39]:
print(f"The estimated mean return is: {estimates[0] : 0.8f} vs {mu : 0.8f}")
print(f"The estimated omega value is: {estimates[1] : 0.8f} vs {w : 0.8f}")
print(f"The estimated alpha value is: {estimates[2] : 0.8f} vs {a : 0.08f}")
print(f"The estimated beta values is: {estimates[3] : 0.8f} vs {b : 0.08f}")

The estimated mean return is:  0.00049600 vs  0.00055461
The estimated omega value is:  0.00000238 vs  0.00000100
The estimated alpha value is:  0.08351405 vs  0.08500000
The estimated beta values is:  0.88917855 vs  0.90500000


<br>

___So much depends upon those initial guesses!___

<br>